## 🧪 ChaCha20 – Using `cryptography` (IETF standard)

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.backends import default_backend
import os

# Key and nonce (fixed nonce length)
key = os.urandom(32)             # 256-bit key
nonce = os.urandom(16)           # 128-bit nonce (required by this implementation)
plaintext = b"Hello from ChaCha20 stream cipher!"

# Encrypt
algorithm = algorithms.ChaCha20(key, nonce)
cipher = Cipher(algorithm, mode=None, backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext)

# Decrypt
decryptor = cipher.decryptor()
recovered = decryptor.update(ciphertext)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)


Ciphertext: 0817ade6b1d9eb54559b364997453f95b705720519984b0c0ae5de9c51dbb8bd7ec1
Recovered: b'Hello from ChaCha20 stream cipher!'


## 🧪 Salsa20 – Using `pycryptodome`

In [2]:
from Crypto.Cipher import Salsa20
import os

# Key and nonce
key = os.urandom(32)              # 256-bit key
nonce = os.urandom(8)             # 64-bit nonce
plaintext = b"Hello from Salsa20 stream cipher!"

# Encrypt
cipher = Salsa20.new(key=key, nonce=nonce)
ciphertext = cipher.encrypt(plaintext)

# Decrypt
cipher_dec = Salsa20.new(key=key, nonce=nonce)
recovered = cipher_dec.decrypt(ciphertext)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)


Ciphertext: 775574866d845e81b574da87bb481055bde3ec106119c6c9eb1b2185bf088fccd2
Recovered: b'Hello from Salsa20 stream cipher!'


## 🧪 Trivium – Toy Python implementation (educational use only)

In [3]:
def trivium_init(key, iv):
    # Trivium uses 80-bit key and IV
    assert len(key) == 10 and len(iv) == 10
    key_bits = sum([[int(b) >> i & 1 for i in range(8)] for b in key], [])
    iv_bits  = sum([[int(b) >> i & 1 for i in range(8)] for b in iv], [])

    state = [0] * 288
    state[:80] = key_bits
    state[93:173] = iv_bits
    state[285:] = [1, 1, 1]

    for _ in range(4 * 288):  # Warm-up phase
        t1 = state[65] ^ (state[90] & state[91]) ^ state[92] ^ state[170]
        t2 = state[161] ^ (state[174] & state[175]) ^ state[176] ^ state[263]
        t3 = state[242] ^ (state[285] & state[286]) ^ state[287] ^ state[68]
        state = [t3] + state[:92] + [t1] + state[93:176] + [t2] + state[177:287]
    return state

def trivium_keystream(state, length):
    stream = []
    for _ in range(length * 8):
        t1 = state[65] ^ state[92]
        t2 = state[161] ^ state[176]
        t3 = state[242] ^ state[287]
        z = t1 ^ t2 ^ t3
        stream.append(z)
        t1 = t1 ^ (state[90] & state[91]) ^ state[170]
        t2 = t2 ^ (state[174] & state[175]) ^ state[263]
        t3 = t3 ^ (state[285] & state[286]) ^ state[68]
        state = [t3] + state[:92] + [t1] + state[93:176] + [t2] + state[177:287]
    return stream

def xor_bytes(data, stream_bits):
    return bytes(b ^ sum([bit << i for i, bit in enumerate(stream_bits[n*8:n*8+8])])
                 for n, b in enumerate(data))

# Key, IV (10 bytes each = 80 bits)
key = b'1234567890'
iv  = b'abcdefghij'
plaintext = b"Trivium stream cipher example!"

# Initialize and encrypt
state = trivium_init(key, iv)
stream_bits = trivium_keystream(state.copy(), len(plaintext))
ciphertext = xor_bytes(plaintext, stream_bits)

# Decrypt
state = trivium_init(key, iv)
stream_bits = trivium_keystream(state.copy(), len(ciphertext))
recovered = xor_bytes(ciphertext, stream_bits)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)


Ciphertext: 03ec5f853e5fded4abedb8ec9b10d5c217e640eae4df167a453367d64eea
Recovered: b'Trivium stream cipher example!'
